In [1]:
#reading in data sets 
import numpy as np
import pandas as pd
import csv
import re

_1718 =pd.read_csv(r"C:\Users\Derek\Desktop\proj\Disag_2017-18_Data.txt", sep='\t',dtype={'school_code':object, 'name':object, 'grade_span':object, 'subject':object, 'grade':object, 'type':object, 'subgroup':object, 'num_tested':float, 'pct_l1':object, 'pct_l2':object, 'pct_l3':object, 'pct_l4':object, 'pct_l5':object, 'pct_ccr':object, 'pct_glp':object, 'avg_score':float})     
_1617 =pd.read_csv(r"C:\Users\Derek\Desktop\proj\Disag_2016-17_Data.txt", sep='\t',dtype={'school_code':object, 'name':object, 'grade_span':object, 'subject':object, 'grade':object, 'type':object, 'subgroup':object, 'num_tested':float, 'num_l1':object, 'pct_l1':object, 'num_l2':object, 'pct_l2':object, 'num_l3':object, 'pct_l3':object, 'num_l4':object, 'pct_l4':object, 'num_l5':object, 'pct_l5':object, 'num_ccr':object, 'pct_ccr':object, 'num_glp':object, 'pct_glp':object})     
_1516 =pd.read_csv(r"C:\Users\Derek\Desktop\proj\Disag_2015-16_Data.txt", sep='\t',dtype={'school_code':object, 'name':object, 'grade_span':object, 'subject':object, 'grade':object, 'type':object, 'subgroup':object, 'num_tested':float, 'num_l1':object, 'pct_l1':object, 'num_l2':object, 'pct_l2':object, 'num_l3':object, 'pct_l3':object, 'num_l4':object, 'pct_l4':object, 'num_l5':object, 'pct_l5':object, 'num_ccr':object, 'pct_ccr':object, 'num_glp':object, 'pct_glp':object})     
_1415 =pd.read_csv(r"C:\Users\Derek\Desktop\proj\Disag_2014-15_Data.txt", sep='\t',dtype={'school_code':object, 'name':object, 'grade_span':object, 'subject':object, 'grade':object, 'type':object, 'subgroup':object, 'num_tested':float, 'num_l1':object, 'pct_l1':object, 'num_l2':object, 'pct_l2':object, 'num_l3':object, 'pct_l3':object, 'num_l4':object, 'pct_l4':object, 'num_l5':object, 'pct_l5':object, 'num_ccr':object, 'pct_ccr':object, 'num_glp':object, 'pct_glp':object})     
_1314 =pd.read_csv(r"C:\Users\Derek\Desktop\proj\Disag_2013-14_Data.txt", sep='\t',dtype={'school_code':object, 'name':object, 'grade_span':object, 'subject':object, 'grade':object, 'type':object, 'subgroup':object, 'num_tested':float, 'num_l1':object, 'pct_l1':object, 'num_l2':object, 'pct_l2':object, 'num_l3':object, 'pct_l3':object, 'num_l4':object, 'pct_l4':object, 'num_l5':object, 'pct_l5':object, 'num_ccr':object, 'pct_ccr':object, 'num_glp':object, 'pct_glp':object})     
school=pd.read_csv(r"C:\Users\Derek\Desktop\proj\school.txt", sep='\t',dtype={'school_code':str,'grade_span':object,'school_name':object})

In [2]:
#adding years to the dataset
_1718['year']=2017
_1617['year']=2016
_1516['year']=2015
_1415['year']=2014
_1314['year']=2013

In [3]:
#set all data on top of each other

data=_1718.append(_1617.append(_1516.append(_1415.append(_1314))))
data=data.replace(['*','<'],'0')
data.fillna(0 ,inplace = True)
school.fillna(0 ,inplace = True)

#dropped columns i didnt want

data=data.drop(['grade_span','avg_score'],axis=1)

In [4]:
#merged on school_code and made prefix numbers to merge on districts

data1=pd.merge(data, school, on=['school_code'])
data1['school_prefix_number']=data1['school_code'].astype(str).str[:3]
data1['school_suffix_number']=data1['school_code'].astype(str).str[3:6]
data1['school_prefix_number2']=data1['school_prefix_number'].astype(str).str[:2]
data1.fillna('none',inplace=True)

#get rid of NC- prefixes, they are SBE districts that we don't need

data1=data1[data1['school_prefix_number'] !='NC-' ]

# data takes to long to run with all subgroups and types so I narrowed it down to just all

data1=data1[data1['subgroup'] =='ALL' ]
data1=data1[data1['type'] =='ALL' ]
data1=data1[data1['grade'] =='ALL' ]

In [5]:
#reading in district data sets

public=pd.read_csv(r"C:\Users\Derek\Desktop\proj\public.txt",sep='\t',dtype={'school_prefix_number':object,'district_name':str,'phone':object})
charter=pd.read_csv(r"C:\Users\Derek\Desktop\proj\charter.csv", dtype={'district_name':str,'school_prefix_number':object,'phone':object})
county=pd.read_csv(r"C:\Users\Derek\Desktop\proj\county.txt",sep='\t',dtype={'district_name':str,'City':object,'County name':object})

#appended charter and public school district datasets together

district=public.append(charter)

#create a way to make district names match county district names 

district['district_name']=district['district_name'].str.replace(r"(Schools)",r"School District ")
district['district_name']=district['district_name'].str.replace(r"(Academy)",r"Academy School District ")

# removed trailing spaces

district['district_name']=district['district_name'].map(lambda x: x.strip())
county['district_name']=county['district_name'].map(lambda x: x.strip())


In [19]:
#merged districts and county data on district names and created prefix2 for code for all distinct counties

district1=pd.merge(district, county, on=['district_name'],how='left')
district1['school_prefix_number2']=district1['school_prefix_number'].astype(str).str[:2]


In [20]:
#create district 2 dataframe with the unique county school prefixes and the name of those counties.

district2=district1
district2=district2.drop(['phone','City ','school_prefix_number'],axis=1)
district1=district1.drop(['County name','district_name'],axis=1)
district2=district2.dropna(subset=['County name'])
district2=district2.drop_duplicates(subset=['school_prefix_number2'])


In [8]:
#merging counties onto the districts for all districts, 

#county prefix 99 is still a little messed up it has multiple counties on it.

district3=pd.merge(district1,district2,on=['school_prefix_number2'])
district3.to_csv('district.csv', sep=',', index=False)

In [15]:
#left merged data1 on to district to put each school into their county

data2=pd.merge(data1,district3, on=['school_prefix_number2'],how='left')
data2.loc[data2['school_prefix_number_x'] == '590', 'County name'] = 'McDowell'
data2.loc[data2['school_prefix_number_x'] == '995', 'County name'] = 'Yancey'
data2.loc[data2['school_prefix_number_x'] >= '996', 'County name'] = 'State'

data2.loc[data2['school_prefix_number_x'] == '590', 'district_name'] = 'McDowell County School District'
data2.loc[data2['school_prefix_number_x'] == '995', 'district_name'] = 'Yancey County School District'
data2.loc[data2['school_prefix_number_x'] >= '996', 'district_name'] = 'State School District'
data2['school_prefix_number']=data2['school_prefix_number_x']
data2=data2.drop(['grade_span','school_prefix_number_x','school_prefix_number_y','City ','phone','name','num_l1','num_l2','num_l3','num_l4','num_l5'],axis=1)      

In [16]:
#seeing if there is any missing counties 

man=data2[data2['County name']=='']
man['County name'].describe()

count     0
unique    0
Name: County name, dtype: int64

In [17]:
data2=data2[data2['school_prefix_number'] !='298' ]
data2=data2[data2['school_suffix_number'] !='LEA' ]

#put in city districts

data2.loc[data2['school_prefix_number'] == '111', 'district_name'] = 'Asheville City School District'
data2.loc[data2['school_prefix_number'] == '132', 'district_name'] = 'Kannapolis City School District'
data2.loc[data2['school_prefix_number'] == '181', 'district_name'] = 'Hickory City School District'
data2.loc[data2['school_prefix_number'] == '182', 'district_name'] = 'Newton Conover City School District'
data2.loc[data2['school_prefix_number'] == '241', 'district_name'] = 'Whiteville City School District'
data2.loc[data2['school_prefix_number'] == '291', 'district_name'] = 'Lexington City School District'
data2.loc[data2['school_prefix_number'] == '292', 'district_name'] = 'Thomasville City School District'
data2.loc[data2['school_prefix_number'] == '421', 'district_name'] = 'Roanoke Rapids City School District'
data2.loc[data2['school_prefix_number'] == '422', 'district_name'] = 'Weldon City School District'
data2.loc[data2['school_prefix_number'] == '491', 'district_name'] = 'Mooresville Graded School District'
data2.loc[data2['school_prefix_number'] == '681', 'district_name'] = 'Chapel Hill-Carrboro City School District'
data2.loc[data2['school_prefix_number'] == '761', 'district_name'] = 'Asheboro City School District'
data2.loc[data2['school_prefix_number'] == '821', 'district_name'] = 'Clinton City School District'
data2.loc[data2['school_prefix_number'] == '861', 'district_name'] = 'Elkin City School District'
data2.loc[data2['school_prefix_number'] == '862', 'district_name'] = 'Mount Airy City School District'

data2=data2.drop_duplicates(subset=['year','school_name','pct_glp','num_tested','subject'])

In [18]:
data2=data2.replace('<5','5')
data2=data2.replace('>95','95')

data2['num_tested']=data2['num_tested'].astype(int)
data2['pct_ccr']=data2['pct_ccr'].astype(float)
data2['pct_glp']=data2['pct_glp'].astype(float)
data2=data2.drop(['pct_l1','pct_l2','pct_l3','pct_l4','pct_l5','num_ccr','num_glp','school_suffix_number','school_prefix_number2','school_prefix_number'],axis=1)      
data2.to_csv('data.csv', sep=',', index=False)